In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
# Importing neccesary packages.

import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator

#

from scipy import interp
import math
from scipy.stats import norm
from scipy import stats

#

import warnings
warnings.filterwarnings('ignore') # Disabling warnimgs for clearer outputs
pd.options.display.max_columns = 50 # Pandas option to increase max number of columns to display
plt.style.use('ggplot') # Setting default plot style

In [ ]:
vtrain=pd.read_csv('/kaggle/input/titanic/train.csv')
vtest=pd.read_csv('/kaggle/input/titanic/test.csv')


In [ ]:
#see data summary
vtrain.info()
vtrain.isnull().sum()
vtest.info()
vtest.isnull().sum()

In [ ]:
# Merging visualization datasets.
vtrain.drop('PassengerId', axis=1, inplace=True)
vtest.drop('PassengerId', axis=1, inplace=True)
merged = pd.concat([vtrain, vtest], sort=False).reset_index(drop=True)
# Checking merged shape
display(merged.shape)

In [ ]:
merged.info()

In [ ]:
train_corr = vtrain[['Survived', 'Age', 'Fare'
                      ]].corr(method='spearman').abs().unstack().sort_values(
                          kind='quicksort', ascending=False).reset_index()


train_corr.rename(columns={
    'level_0': 'Feature A',
    'level_1': 'Feature B',
    0: 'Correlation Coefficient'
},
                  inplace=True)
train_corr
train_corr[(train_corr['Feature A'] == 'Survived')].style.background_gradient(
    cmap='summer_r')

In [ ]:
#detecting missing values
import seaborn as sns
%matplotlib inline
fig, ax = plt.subplots(ncols=2, figsize=(20, 6))
sns.heatmap(vtrain.isnull(),
            yticklabels=False,
            cbar=False,
            cmap='magma',
            ax=ax[0])
sns.heatmap(vtest.isnull(),
            yticklabels=False,
            cbar=False,
            cmap='magma',
            ax=ax[1])

ax[0].set_title('Train Data Missing Values')
ax[1].set_title('Test Data Missing Values')
plt.xticks(rotation=90)
plt.show()

In [ ]:
def status(feature):
    print('Processing', feature, ': DONE')
    print(f'Shape after processing {combined.shape}')
    print('*' * 40)

In [ ]:
def get_combined_data():
    # Reading train data
    train = pd.read_csv('/kaggle/input/titanic/train.csv')

    # Reading test data
    test = pd.read_csv('/kaggle/input/titanic/test.csv')

    # extracting the targets from the training data
    targets = train.Survived

    # Merging train data and test data for future feature engineering
    combined = train.append(test)
    combined.reset_index(inplace=True, drop=True)

    return combined

In [ ]:
def process_family():

    global combined
    # Introducing a new feature : The size of families (including the passenger)
    combined['FamilySize'] = combined['Parch'] + combined['SibSp'] + 1

    # Introducing another feature based on the family size
    combined['Alone'] = combined['FamilySize'].map(lambda s: 1
                                                   if s == 1 else 0)

    # These two below are optional, it didn't help with the model...
    #combined['SmallFamily'] = combined['FamilySize'].map(lambda s: 1 if 2 <= s <= 4 else 0)
    #combined['LargeFamily'] = combined['FamilySize'].map(lambda s: 1 if 5 <= s else 0)

    status('Family')
    return combined

In [ ]:
def family_survival():    
    global combined

    # A function working on family survival rate using last names and ticket features

    # Extracting surnames
    combined['Last_Name'] = combined['Name'].apply(
        lambda x: str.split(x, ",")[0])

    # Adding new feature: 'Survived'
    default_survival_rate = 0.5
    combined['Family_Survival'] = default_survival_rate

    for grp, grp_df in combined[[
            'Survived', 'Name', 'Last_Name', 'Fare', 'Ticket', 'PassengerId',
            'SibSp', 'Parch', 'Age', 'Cabin'
    ]].groupby(['Last_Name', 'Fare']):
        
        if (len(grp_df) != 1):
            # A Family group is found.
            for ind, row in grp_df.iterrows():
                smax = grp_df.drop(ind)['Survived'].max()
                smin = grp_df.drop(ind)['Survived'].min()
                passID = row['PassengerId']
                if (smax == 1.0):
                    combined.loc[combined['PassengerId'] ==
                                 passID, 'Family_Survival'] = 1
                elif (smin == 0.0):
                    combined.loc[combined['PassengerId'] ==
                                 passID, 'Family_Survival'] = 0

    for _, grp_df in combined.groupby('Ticket'):
        if (len(grp_df) != 1):
            for ind, row in grp_df.iterrows():
                if (row['Family_Survival'] == 0) | (
                        row['Family_Survival'] == 0.5):
                    smax = grp_df.drop(ind)['Survived'].max()
                    smin = grp_df.drop(ind)['Survived'].min()
                    passID = row['PassengerId']
                    if (smax == 1.0):
                        combined.loc[combined['PassengerId'] ==
                                     passID, 'Family_Survival'] = 1
                    elif (smin == 0.0):
                        combined.loc[combined['PassengerId'] ==
                                     passID, 'Family_Survival'] = 0

    status('FamilySurvival')
    return combined


In [ ]:
def get_titles():

    title_dictionary = {
        'Capt': 'Dr/Clergy/Mil',
        'Col': 'Dr/Clergy/Mil',
        'Major': 'Dr/Clergy/Mil',
        'Jonkheer': 'Honorific',
        'Don': 'Honorific',
        'Dona': 'Honorific',
        'Sir': 'Honorific',
        'Dr': 'Dr/Clergy/Mil',
        'Rev': 'Dr/Clergy/Mil',
        'the Countess': 'Honorific',
        'Mme': 'Mrs',
        'Mlle': 'Miss',
        'Ms': 'Mrs',
        'Mr': 'Mr',
        'Mrs': 'Mrs',
        'Miss': 'Miss',
        'Master': 'Master',
        'Lady': 'Honorific'
    }

    # Extract the title from names
    combined['Title'] = combined['Name'].map(
        lambda name: name.split(',')[1].split('.')[0].strip())

    # Mapping titles
    combined['Title'] = combined.Title.map(title_dictionary)
    status('Title')
    return combined

In [ ]:
def process_names():
    global combined
    # Cleaning the Name variable
    combined.drop('Name', axis=1, inplace=True)

    # Encoding names as dummy variables
    titles_dummies = pd.get_dummies(combined['Title'], prefix='Title')
    combined = pd.concat([combined, titles_dummies], axis=1)

    # Removing the title variable after getting dummies
    combined.drop('Title', axis=1, inplace=True)

    status('names')
    return combined

In [ ]:
def process_age():
    global combined
    # A function that fills the missing values of the Age variable
    combined['Age'] = combined.groupby(
        ['Pclass', 'Sex'])['Age'].apply(lambda x: x.fillna(x.median()))
    status('Age')
    return combined

In [ ]:
def age_binner():    
    global combined
    # Ranging and grouping Ages
    #bins = [0, 2, 18, 35, 65, np.inf]
    names = ['less2', '2-18', '18-35', '35-65', '65plus']

    #combined['AgeBin'] = pd.cut(combined['Age'], bins, labels=names)
    combined['AgeBin'] = pd.qcut(combined['Age'],q = 5, labels = names)
    age_dummies = pd.get_dummies(combined['AgeBin'], prefix='AgeBin')
    combined = pd.concat([combined, age_dummies], axis=1)
    combined.drop('AgeBin', inplace=True, axis=1)
    combined.drop('Age', inplace=True, axis=1)
    status('Age Bins')
    return combined

In [ ]:
def process_fares():
    global combined

    # Filling missing values in fare

    combined['Fare'] = combined.groupby(
        ['Pclass', 'Sex'])['Fare'].apply(lambda x: x.fillna(x.median()))
    status('fare')
    return combined

In [ ]:
def process_fare_bin(onehot='None'):

    global combined
    # Ranging and grouping Fare using historical data
    bins = [-1, 7.91, 14.454, 31, 99, 250, np.inf]
    names = [0, 1, 2, 3, 4, 5]

    combined['FareBin'] = pd.cut(combined['Fare'], bins,
                                 labels=names).astype('int')
    if onehot == 'yes':
        farebin_dummies = pd.get_dummies(combined['FareBin'], prefix='FareBin')
        combined = pd.concat([combined, farebin_dummies], axis=1)
        combined.drop('FareBin', inplace=True, axis=1)
        combined.drop('Fare', inplace=True, axis=1)
    elif onehot == 'both':
        farebin_dummies = pd.get_dummies(combined['FareBin'], prefix='FareBin')
        combined = pd.concat([combined, farebin_dummies], axis=1)
        combined.drop('FareBin', inplace=True, axis=1)
    else:
        combined.drop('Fare', inplace=True, axis=1)

    status('FareBin')
    return combined

In [ ]:
def scale_fare():

    # A definition for scaling fare values

    from scipy.stats import skew, boxcox_normmax, norm
    from scipy.special import boxcox1p
    global combined
    combined['Fare'] = boxcox1p(combined['Fare'],
                                boxcox_normmax(combined['Fare'] + 1))
    status('NFareBin')
    return combined

In [ ]:
def process_embarked():
    global combined
    # Filling missing embarked values with the most frequent one
    combined.Embarked.fillna(combined.Embarked.mode()[0], inplace=True)
    # One hot encoding
    embarked_dummies = pd.get_dummies(combined['Embarked'], prefix='Embarked')
    combined = pd.concat([combined, embarked_dummies], axis=1)
    combined.drop('Embarked', axis=1, inplace=True)
    status('Embarked')
    return combined

In [ ]:
def process_cabin():    
    global combined
    # Replacing missing cabins with M (for Missing)
    combined['Cabin_Informed'] = [
        1 if pd.notnull(cab) else 0 for cab in combined['Cabin']
    ]
    combined.Cabin.fillna('M', inplace=True)

    # Mapping each Cabin value with the cabin letter
    combined['Deck'] = combined['Cabin'].map(lambda c: c[0])

    combined['Deck'].replace('T', 'A', inplace=True)

    # One hot encoding ...
    cabin_dummies = pd.get_dummies(combined['Deck'], prefix='Deck')
    combined = pd.concat([combined, cabin_dummies], axis=1)

    combined.drop('Cabin', axis=1, inplace=True)
    combined.drop('Deck', axis=1, inplace=True)
    status('Cabin')
    return combined

In [ ]:
def process_sex():
    global combined
    # Mapping string values with numerical ones
    combined['Sex'] = combined['Sex'].map({'male': 1, 'female': 0})
    status('Sex')
    return combined

In [ ]:
def process_pclass():
    global combined
    # One hot encoding pclass into 3 categories:
    pclass_dummies = pd.get_dummies(combined['Pclass'], prefix='Pclass')

    # Adding dummy variables to main set
    combined = pd.concat([combined, pclass_dummies], axis=1)

    # Removing redundant 'Pclass'
    combined.drop('Pclass', axis=1, inplace=True)

    status('Pclass')
    return combined



In [ ]:
def process_ticket():

    global combined

    # A function that extracts each prefix of the ticket, returns 'Unknown' if no prefix (i.e the ticket is a digit)
    def cleanTicket(ticket):
        ticket = ticket.replace('.', '')
        ticket = ticket.replace('/', '')
        ticket = ticket.split()
        ticket = map(lambda t: t.strip(), ticket)
        ticket = list(filter(lambda t: not t.isdigit(), ticket))
        if len(ticket) > 0:
            return ticket[0]
        else:
            return 'Unknown'

    # Extracting dummy variables from tickets:

    combined['Ticket'] = combined['Ticket'].map(cleanTicket)
    tickets_dummies = pd.get_dummies(combined['Ticket'], prefix='Ticket')
    combined = pd.concat([combined, tickets_dummies], axis=1)
    combined.drop('Ticket', inplace=True, axis=1)

    status('Ticket')
    return combined

# Housekeeping
This function is for cleaning the redundant features after extracting useful information from them. The cabin, passengerId, last name, ticket and ofc our target survived is going to be dropped. These features did their job on feature engineering and we can leave them in peace now.

In [ ]:
def dropper():
    global combined

    combined.drop('Cabin', axis=1, inplace=True)
    combined.drop('PassengerId', inplace=True, axis=1)
    combined.drop('Last_Name', inplace=True, axis=1)
    combined.drop('Survived', inplace=True, axis=1)
    combined.drop('Ticket', inplace=True, axis=1)

    return combined

# Feeding the Machine
Ok this is our command room, this is where I control the switches for feature engineering(aka commenting some functions out or combine them :D) Well, this was pretty trial and error methodology for me. Tried couple combinations of feature engineering and tried to find best fit on the models. Here is what I came up with.

In [ ]:
# Executing definitions to process data
combined = get_combined_data()  # For merging train test data
combined = family_survival()  # For creating family survival feature
combined = process_family()  # For creating Family size feature
combined = get_titles()  # For extracting titles
combined = process_names()  # For one hot encoding titles
combined = process_age()  # For imputing missing age values
combined = age_binner()  # For grouping and hot encoding age ranges
combined = process_fares()  # For imputing fares
# For grouping and label encoding fares, can use 'both' for keeping age with dummies or yes for just one hot
combined = process_fare_bin(onehot='no')
# combined =scale_fare() # For scaling age values
combined = process_embarked()  # For imputing embarked and one hot encoding
# combined = process_cabin() # For extracting deck info from cabins
combined = process_sex()  # For label encoding sex
# combined = process_pclass() # For one hot encoding pclass
# combined = process_ticket() # For extracting ticket info
combined = dropper()  # For dropping not needed features
print(
    f'Processed everything. Missing values left: {combined.isna().sum().sum()}'
)

# Double Check
Ok, this is the part where we're going make sure our data is ready for the modelling. So, let's start with checking how's our data looking. Everyting seems in order...

In [ ]:
# Assigning engineered data to inspection variable.
v_merged = combined.copy()
v_merged['Survived'] = vtrain['Survived']
v_merged.head()

In [ ]:
v_merged.describe()

# Correlation Matrix
Alright one last table I'm going to check is correlation matrix. So we can spot linear relations between features. Especially the ones which effects survival rate. It seems Sex, the Mr. title and family survival ratio is most related features to survival.

Well... That's it then let's continue with modelling now!

In [ ]:
# Display numerical correlations between features on heatmap
sns.set(font_scale=1.1)
correlation_train = v_merged.corr()
mask = np.triu(correlation_train.corr())
plt.figure(figsize=(18, 15))
sns.heatmap(correlation_train,
            annot=True,
            fmt='.1f',
            cmap='coolwarm',
            square=True,
            mask=mask,
            linewidths=1)

plt.show()


# Modelling
Since preprocessing done we are ready for training our models. We start with loading packages and splitting our transformed data so we have 22 features and and 891 observations to train our estimators. Our test set has 418 observations to make predictions.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, learning_curve, cross_validate, train_test_split, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, plot_roc_curve, auc
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE

import xgboost as xgb
import lightgbm as lgb
from mlxtend.plotting import plot_decision_regions

In [ ]:
def recover_train_test_target():
    global combined

    y = pd.read_csv('/kaggle/input/titanic/train.csv', usecols=['Survived'])['Survived']
    X = combined.iloc[:len(vtrain)]
    X_test = combined.iloc[len(vtrain):]

    return X, X_test, y


X, X_test, y = recover_train_test_target()

In [ ]:
print(f'X shape: {X.shape}')
print(f'y shape: {y.shape}')
print(f'X_test shape: {X_test.shape}')

# Model Selection
I choose couple models to try, mostly ensemble models with boosting but we also have some linear models like Logistic Regression. I tuned some these models with Optuna but I'm not going to include these meta parts on this notebook.

In [ ]:
# Models

cv = StratifiedKFold(10, shuffle=True, random_state=42)

def model_check(X, y, estimators, cv):
    model_table = pd.DataFrame()
    x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)
    row_index = 0
    for est in estimators:

        MLA_name = est.__class__.__name__
        model_table.loc[row_index, 'Model Name'] = MLA_name
        #    model_table.loc[row_index, 'MLA Parameters'] = str(est.get_params())

        est.fit(x_train, y_train)
        
        model_table.loc[row_index, 'Train Accuracy Mean'] = est.score(x_train, y_train)
        model_table.loc[row_index, 'Test Accuracy Mean'] = est.score(x_test, y_test)

        row_index += 1

        model_table.sort_values(by=['Test Accuracy Mean'],
                            ascending=False,
                            inplace=True)

    return model_table

rf = RandomForestClassifier(criterion='gini',
                            n_estimators=1750,
                            max_depth=7,
                            min_samples_split=6,
                            min_samples_leaf=6,
                            max_features='auto',
                            oob_score=True,
                            random_state=42,
                            n_jobs=-1,
                            verbose=0)

lg = lgb.LGBMClassifier(max_bin=4,
                        num_iterations=550,
                        learning_rate=0.0114,
                        max_depth=3,
                        num_leaves=7,
                        colsample_bytree=0.35,
                        random_state=42,
                        n_jobs=-1)

xg = xgb.XGBClassifier(
    n_estimators=2800,
    min_child_weight=0.1,
    learning_rate=0.002,
    max_depth=2,
    subsample=0.47,
    colsample_bytree=0.35,
    gamma=0.4,
    reg_lambda=0.4,
    random_state=42,
    n_jobs=-1,
)

sv = SVC(probability=True)

logreg = LogisticRegression(n_jobs=-1, solver='newton-cg')

gb = GradientBoostingClassifier(random_state=42)

gnb = GaussianNB()

mlp = MLPClassifier(random_state=42)


In [ ]:
estimators = [rf, lg, xg, gb, sv, logreg, gnb, mlp]

> Model Results
Hmm... It seems our top classifiers are getting similar results and they are mostly ensemble models. Our basic Logistic Regression did pretty well too! Gradient Boosting Classifier seems effected by high variance, we might try to fix it by hyperparameter tuning but I'd say we already have good number of decent estimators so we can continue with them.

In [ ]:
raw_models = model_check(X, y, estimators, cv)
display(raw_models.style.background_gradient(cmap='summer_r'))

In [ ]:
# Plotting model performances
# model_barplot(raw_models, 32)

# Submission

In [ ]:
rand_model_full_data = rf.fit(X, y)
print(accuracy_score(y, rand_model_full_data.predict(X)))
y_pred = rand_model_full_data.predict(X_test)

In [ ]:
test_df = pd.read_csv('/kaggle/input/titanic/test.csv')
submission_df = pd.DataFrame(columns=['PassengerId', 'Survived'])
submission_df['PassengerId'] = test_df['PassengerId']
submission_df['Survived'] = y_pred
submission_df.to_csv('submission.csv', header=True, index=False)
submission_df.head(10)